## Paper

https://arxiv.org/pdf/1812.07627.pdf

## Repository

https://github.com/kiankd/corel2019

## Silhouette Loss

https://platform.ai/blog/page/11/the-silhouette-loss-function-metric-learning-with-a-cluster-validity-index/

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [54]:
# Helper functions for dealing with AR Loss
def get_armask(shape, labels):
    if labels.dtype != tf.int32:
        raise Exception("Labels must be a LongTensor with dtype=int32!")

    mask = tf.zeros(shape)
    arr = tf.range(0, shape[0], 1.0)

    # want to maximize similarity to the correct classes, so this is negative.
    mask = mask -1.
    return mask

def arloss(attraction_tensor, repulsion_tensor, lam):
    # combine up everything to accumulate across the entire batch
    loss_attraction = tf.reduce_sum(attraction_tensor)
    loss_repulsion = tf.reduce_sum(repulsion_tensor)
    arloss = (lam * loss_attraction) + ((1. - lam) * loss_repulsion)
    return arloss / tf.cast(attraction_tensor.shape[0], 'float32')

In [35]:
# Gaussian-COREL combined loss function
def GaussianARLoss(inputs):

    labels = inputs[:, :1]
    labels = tf.cast(labels, dtype='int32')
    embeddings =  tf.cast(inputs[:, 1:], dtype='float32')
    lam=0.5
    mask = get_armask(embeddings.shape, labels)

    # in this case, use standard LogSoftmax, without AR.
    if lam == 0.5:
        softmax_predictions = tf.nn.log_softmax(embeddings, axis=1)
        loss_tensor = mask * softmax_predictions
        return tf.reduce_mean(loss_tensor)

    # otherwise, do it in the slightly less numerically stable way.
    attraction_tensor = mask * predictions * self.lam
    repulsion_tensor = tf.exp(predictions)
    repulsion_tensor = tf.log(tf.reduce_sum(repulsion_tensor, axis=1) + 1e-10) * (1. - lam)
    return arloss(attraction_tensor, repulsion_tensor, lam)

### Gausian-COREL loss

In [55]:
v = tf.constant([[1.0, 2.0, 2.0], [2.0, 3.0, 4.0], [1.0, 1.5, 2.0], 
                 [1.0, 1.0, 2.0], [3.0, 1.0, 4.0], [3.0, 1.0, 2.0]]) # [2, 3.0, 4.0] [1, 1.0, 2.0]
tl_loss = GaussianARLoss(v)
sess=tf.Session() 
sess.run(tl_loss)

0.9009327

### Cosine-COREL loss

In [52]:
def CosineARLoss(inputs):
    labels = inputs[:, :1]
    labels = tf.cast(labels, dtype='int32')
    embeddings =  tf.cast(inputs[:, 1:], dtype='float32')
    lam=0.5
    
    mask = get_armask(embeddings.shape, labels)

    # make the attractor and repulsor, mask them!
    attraction_tensor = mask * embeddings
    repulsion_tensor = (mask + 1.0) * embeddings

    # now, apply the special cosine-COREL rules, taking the argmax and squaring the repulsion
    repulsion_tensor = tf.reduce_max(repulsion_tensor, axis=1)
    repulsion_tensor = repulsion_tensor ** 2

    return arloss(attraction_tensor, repulsion_tensor, lam)

In [53]:
v = tf.constant([[1.0, 2.0, 2.0], [2.0, 3.0, 4.0], [1.0, 1.5, 2.0], 
                 [1.0, 1.0, 2.0], [3.0, 1.0, 4.0], [3.0, 1.0, 2.0]]) # [2, 3.0, 4.0] [1, 1.0, 2.0]
tl_loss = CosineARLoss(v)
sess=tf.Session() 
sess.run(tl_loss)

-2.125